---
title: Results comparison to SLAMMER 
bibliography: references.bib
sidebar: main
---

In [ ]:
#| echo: false
from pathlib import Path
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pyslammer as slam
import scipy.stats.mstats as ssmstat
from cycler import cycler
from great_tables import GT, md, html
plt.style.use(slam.psfigstyle)

PySLAMMER's rigid, decoupled, and coupled analysis methods are intended to produce sliding block analysis results that match the legacy SLAMMER results. This is an important feature for sliding block displacements, which are used as a performance index (as opposed to providing a direct prediction of actual slope displacement) in practice. Equivalence with legacy results allows new results to be interpreted with reference to historical analyses and experience.

## Approach

To demonstrate pySLAMMER's equivalence to SLAMMER, we performed several sliding block analyses across a broad parametric space. The three main categories of parameters studied were ground motion, analysis method, and analysis options.

### Ground motion

We used the motions from pySLAMMER's built-in sample ground motion suite at several scales to capture a reasonable breadth of the familiar key engineering ground motion characteristics (frequency, amplitude, and duration). The acceleration response spectra for the input motion suite are shown in @fig-resp. Additional details on the ground motions are provided on the [ground motion suite page](ground_motions.qmd).

In [ ]:
#| code-fold: true
#| label: fig-resp
#| fig-align: center
#| fig-cap: Acceleration response spectra for input ground motions

# Navigate to ground motion suite response spectra files
current_dir = os.getcwd()
folder_path = Path(current_dir).resolve().parents[1] / "verification_files" / "pySLAMMER_suite_response_spectra"
csv_files = list(folder_path.glob("*.csv"))

# Read each CSV file into a DataFrame and store them in a list
freq_index = 0
resp_index = 1
spectra = {}
for csv_file in csv_files:
    data = np.loadtxt(csv_file, delimiter=",", skiprows=2)
    # convert response from cm/s^2 to g's
    data[:, resp_index] = data[:, resp_index] / 981
    spectra[csv_file.name.strip(".csv")] = data

# Initialize the plot
fig, ax = plt.subplots()
ax.set_prop_cycle(cycler(color=plt.cm.tab20.colors))
for motion in spectra:
    ax.plot(
        1/spectra[motion][:, freq_index],
        spectra[motion][:, resp_index],
        label=motion,
        linewidth=0.5,
    )

ax.text(0.012, 2.75, "5% damping")
# Add labels, legend, and grid
ax.set_xlabel("Period (s)")
ax.set_ylabel("Spectral Acceleration (g)")
ax.set_title("Response Spectra")
ax.set_xscale("log")
ax.set_ylim(0,3)
ax.set_xlim(0.01,100)

ax.legend(
    loc="center left", 
    bbox_to_anchor=(0.6, 0.6), 
    fontsize="x-small", 
    title="Ground Motion", 
    title_fontsize="medium",
    frameon=False
)

plt.show()

### Analysis methods

The three rigorous analysis methods that SLAMMER performs are the Rigid, Decoupled, and Coupled methods. Each of these methods, which are briefly described below, are implemented in pySLAMMER and included in this comparison.

Rigid Block Analysis

:   Conceptualized by Whitman in 1963 and further developed by Newmark in 1965 (@marcuson1994, @newmark1965), rigid block analysis models a potential landslide mass as a rigid mass on an inclined plane base with a perfectly plastic frictional interface. The rigid block motion matches the base motion exactly until the acceleration of the base exceeds some critical value (the yield acceleration, $k_y$). Once this critical $k_y$ value is reached, the block's acceleration remains constant $k_y$, resulting in relative velocity and displacement between block and base. The relative velocity is calculated by integrating the difference between the block and base accelerations. The displacement accumulated by the block moving down the ramp is calculated by integrating the relative velocity. Sliding stops (i.e., block motion again matches base motion exactly) when the relative velocity reaches zero.

Decoupled Analysis

:   Landslide materials are, of course, not rigid. Except for very shallow, stiff slide masses, the rigid block model does a poor job of approximating the dynamics of a co-seismic landslide system. The decoupled method was developed to provide some way of accounting for the deformation of the slope mass due to shaking (@seed1966, @makdisi1978) . It consists of two distinct (or decoupled, if you will) calculations: *dynamic response* and *rigid sliding*. During the dynamic response phase, the possibility of sliding is ignored while the slope response to strong ground motion is calculated. The average internal acceleration of the slope mass during this first phase is then used as the base input acceleration for the second phase (rigid sliding) which is simply a rigid block analysis.

Coupled Analysis

:   As indicated by the name, coupled analysis takes the two separate calculations from the decoupled analysis and performs them simultaneously. @chopra1991 introduced a model for earthquake-induced sliding of concrete gravity dams that considered the dynamic response of the dam during sliding. @rathje1999 modified the procedure and applied it to earth structures. With coupled sliding, the sliding mass's dynamic response is calculated for both sliding and non-sliding conditions. Sliding stops when the relative velocity of the sliding mass reaches zero. The stop of sliding can introduce an abrupt change in acceleration applied to the sliding mass. Because the dynamic response is being calculated continually through the analysis, the approach to identifying the timing of these abrupt changes will affect potential subsequent sliding events.

Dynamic Response

:   The Decoupled and Coupled methods both require the calculation of the slope's dynamic response. The two methods for dynamic response in SLAMMER, which have been carried over to pySLAMMER are *linear elastic* and *equivalent linear*.

### Analysis options

SLAMMER allows users to include a constant $k_y$ value or a variable $k_y$ that changes with accumulated sliding displacement. The $k_y$ -- displacement relationship is stepwise with a table of paired values. This feature provides a rough means of approximating post-peak residual strength. [^1]

[^1]: pySLAMMER includes additional options for variable yield acceleration, but only stepwise variation is applicable to comparison with SLAMMER

The dynamic response of the system (applicable to decoupled and coupled analyses) is calculated using either *linear elastic* or *equivalent linear* assumptions. The minimum input parameters needed for the linear elastic analyses are a damping ration, the slope height, and the shear wave velocity of the material above and below the slip surface. For equivalent linear analysis, a reference strain parameter is also needed. Although not explicitly documented, inspection of the SLAMMER source code uses Darendeli (2001) modulus reduction and damping curves with a curvature coefficient of 1.

Separate entries for the shear wave velocity of the material above and below the slip surface ($V_s$ and $V_b$, respectively) are used to introduce an equivalent foundation radiation damping into the viscous material damping as described by @lee2004. This happens behind the scenes in SLAMMER by default and cannot be turned off. [^2]

[^2]: If lower total damping than the equivalent foundation radiation damping is needed for some reason (e.g., for comparison with published analyses that did not include this damping mechanism) both SLAMMER and pySLAMMER accept negative values for damping ratio, which can be used to offset the damping applied by the foundation radiation damping.

@tbl-params shows the analysis options and ranges of input values used in the group of simulations used to compare pySLAMMER to

In [ ]:
#| code-fold: true
#| label: tbl-params
#| tbl-cap: Sliding block parameters used for pySLAMMER to SLAMMER comparison analyses.

kykmax = [0.05, 1.0]
tmts = [0.1, 10.0]
height = [0.1, 100]
vs =  [200, 1200]
vsvb =  [0.1, 100]
damp =  [5, 25]
ref_str =  [1, 10]

params = {
    "param": [
        "Methods",
        "Dynamic Resp.",
        "ky/kmax",
        "Tm/Tx *",
        "Slope height (m) *",
        "Slope shear wave vel., Vs (m/s) *",
        "Vs/Vb *",
        "Damping (%) *",
        "Reference strain (%) **"
    ],
    "val": [
        "Rigid, Decoupled, Coupled",
        "Linear elastic, Equivalent linear",
        f"{kykmax[0]} to {kykmax[1]}",
        f"{tmts[0]} to {tmts[1]}",
        f"{height[0]} to {height[1]}",
        f"{vs[0]} to {vs[1]}",
        f"{vsvb[0]} to {vsvb[1]}",
        f"{damp[0]} to {damp[1]}",
        f"{ref_str[0]} to {ref_str[1]}"
    ]
}

# * only applicable to Decoupled and Coupled methods
# ** only applicable to equivalent linear dynamic response

tbl = (
    GT(pd.DataFrame(params))
    .cols_label(
    param=html("Parameter"),
    val=html("Range / Options used")
    )
    .cols_align(align="center", columns=1)
    .tab_source_note("* Only applies to Decouled and Coupled analyses") 
    .tab_source_note("** Only applies to equivalent linear analyses")
)
tbl

## Results

Although the Python code in pySLAMMER essentially uses the same algorithms as the Java code in SLAMMER, some non-zero differences in results should be expected due to minor numerical differences (e.g., number type handling, order of operations) [@ince2012]. However, the results can be considered functionally equivalent if they contain no apparent tendency to diverge and the differences are relatively small. For engineering purposes, sliding block displacements below 0.5 cm can be considered equivalent to zero displacement [@bray2019]. The results of the simulations across the parametric space described in @tbl-params are shown on @fig-comp. The data are plotted on a log-log scale to make minor numerical differences visible near and below the "equivalent zero" displacement threshold. As indicated by the linear regression, the differences between SLAMMER and pySLAMMER results are negligibly small relative to the scales of engineering significance.

In [ ]:
#| code-fold: true
#| label: fig-comp
#| fig-align: center
#| fig-cap: Comparison of pySLAMMER and SLAMMER analysis results. At this scale, the differences are too small to be distinguished visually.

# Load SLAMMER and pySLAMMER results from verification data
import json
import gzip
from matplotlib.patches import FancyArrowPatch

# Load SLAMMER results
with gzip.open('../../verification_files/verification_data/results/slammer_results.json.gz', 'rt') as f:
    slammer_data = json.load(f)

# Load pySLAMMER results (using latest version)
with gzip.open('../../verification_files/verification_data/results/pyslammer_0.2.3_results.json.gz', 'rt') as f:
    pyslammer_data = json.load(f)

# Create comparison dataframe
comparison_data = []

# Create lookup for pySLAMMER results by analysis_id
pyslammer_lookup = {analysis['analysis_id']: analysis for analysis in pyslammer_data['analyses']}

for slammer_analysis in slammer_data['analyses']:
    analysis_id = slammer_analysis['analysis_id']
    
    if analysis_id in pyslammer_lookup:
        pyslammer_analysis = pyslammer_lookup[analysis_id]
        
        # Get method name and convert to title case
        method = slammer_analysis['analysis']['method']
        method_name = method.title() if method != 'rigid' else 'Rigid'
        
        comparison_data.append({
            'analysis_id': analysis_id,
            'Method': method_name,
            'SLAMMER': slammer_analysis['results']['normal_displacement_cm'],
            'pySLAMMER': pyslammer_analysis['results']['normal_displacement_cm']
        })

# Create dataframe
dfp = pd.DataFrame(comparison_data)

# Calculate overall linear regression for statistics display
from scipy.stats import linregress
slope, intercept, r, p, se = linregress(dfp['SLAMMER'], dfp['pySLAMMER'])


plt.close("all")
############
# pySLAMMER v. SLAMMER
############
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.sans-serif"] = ["Arial"]
plt.rcParams["savefig.dpi"] = 600

fig, ax = plt.subplots()



keys = ["Decoupled", "Rigid", "Coupled"]
markers = ["^", "P", "o"]


for i, key in enumerate(keys):
    grp = dfp[dfp["Method"] == key]
    ax.scatter(grp["SLAMMER"], 
               grp["pySLAMMER"], 
               label=key, 
               alpha=0.35,
               marker=markers[i],
               s=20
               )


ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(1e-3, 1e3)
ax.set_ylim(1e-3, 1e3)

ax.set_aspect("equal")

plt.grid()

ax.set_xlabel("SLAMMER displacement (cm)")
ax.set_ylabel("pySLAMMER displacement (cm)")

# Linear regression

ax.plot([1e-3, 1e3], [1e-3, 1e3], color="black", linestyle="--", linewidth=1)

ax.text(
    0.67,
    0.585,
    f"Linear Regression\nSlope = {slope:.2f}\nIntercept = {intercept:.1f} cm\nR$^2$ = {r**2:.3f}",
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment="top",
    bbox=dict(facecolor="white", alpha=1),
)

# Arrow for linear regression
arrow = FancyArrowPatch(
    (0.85, 0.6),  # Start point (near text box)
    (0.75, 0.75),  # End point (middle of plot, on dashed line)
    connectionstyle="angle3,angleA=90,angleB=-45",  # Angle connection style
    transform=ax.transAxes,
    arrowstyle="->",
    mutation_scale=11,
    color="black",
    alpha=1,
    zorder=4
)
ax.add_patch(arrow)

# Engineering range of interest
ax.plot([0,0.5,0.5],[0.5,0.5,0], color="black", linestyle="-", linewidth=1)

# Add text box with regression statistics
ax.text(
    0.475,
    0.145,
    "Engineering \"equivalent zero\"\ndisplacement (0.5 cm)",
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment="top",
    bbox=dict(facecolor="white", alpha=1),
)

# Arrow for linear regression
arrow = FancyArrowPatch(
    (0.65, 0.155),  # Start point (near text box)
    (0.45, 0.245),  # End point (middle of plot, on dashed line)
    connectionstyle="angle3,angleA=90,angleB=-5",  # Angle connection style
    transform=ax.transAxes,
    arrowstyle="->",
    mutation_scale=10,
    color="black",
    alpha=1,
    zorder=4,
)
ax.add_patch(arrow)

# Add legend
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.05, 0.95),
    fontsize=10,
    title="Method",
    title_fontsize="11",
    framealpha=1,
)

plt.show()

## Conclusions

The results show that pySLAMMER is performing the same rigorous analysis methods as SLAMMER. Across a broad array of input parameters, the results are identical or nearly identical. Minor numerical differences between the output of the two programs is to be expected. However, the differences are so small as to be insignificant for engineering purposes, both in research and practice.